In [1]:
#1a
!pip3 install ipywidgets
# We use ipywidgets here to create interactive sliders that let us easily adjust simulation parameters (like satellite count, collision rate, and cleanup rate) and instantly see how those changes affect the Kessler Syndrome model over time

In [5]:
import sys
!{sys.executable} -m pip install --break-system-packages numpy matplotlib moviepy ipywidgets


  Using cached moviepy-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached imageio-2.37.2-py3-none-any.whl.metadata (9.7 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-macosx_10_9_intel.macosx_10_9_x86_64.whl.metadata (1.5 kB)
  Using cached proglog-0.1.12-py3-none-any.whl.metadata (794 bytes)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 11.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 15.6 MB/s  0:00:00 eta 0:00:01
Using cached moviepy-2.2.1-py3-none-any.whl (129 kB)
Using cached imageio-2.37.2-py3-none-any.whl (317 kB)
   ━━━━━━━━

In [2]:
#1b
!pip3 install matplotlib moviepy numpy


In [ ]:
#2a
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# --- Constants ---
GM = 398600  # Gravitational constant times mass of Earth (scaled for sim)
earth_radius = 30
dt = 0.1  # Simulation time step in seconds

# --- Satellite Setup ---
num_sats = 8
sat_radii = np.random.uniform(90, 110, num_sats)
angles = np.linspace(0, 2*np.pi, num_sats, endpoint=False)
sat_positions = np.stack([sat_radii * np.cos(angles), sat_radii * np.sin(angles)], axis=-1)

sat_directions = np.stack([-np.sin(angles), np.cos(angles)], axis=-1)
sat_speeds = np.sqrt(GM / sat_radii)
sat_velocities = sat_directions * sat_speeds[:, None]

# Add slight orbital instability
sat_velocities[0] *= 0.85
sat_colors = ['blue'] * num_sats
sat_sizes = np.ones(num_sats) * 3

# --- Debris ---
debris_positions, debris_velocities = [], []
debris_colors, debris_sizes = [], []

# --- Tracking ---
object_counts = []

# --- Plot Setup ---
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-200, 200)
ax.set_ylim(-200, 200)
ax.set_aspect('equal')
ax.axis('off')

earth = plt.Circle((0, 0), earth_radius, color='blue', alpha=0.3)
ax.add_artist(earth)
scat = ax.scatter([], [], s=[], c=[])

# --- Update Function ---
def update(frame):
    global sat_positions, sat_velocities, sat_colors, sat_sizes
    global debris_positions, debris_velocities, debris_colors, debris_sizes

    # Update satellites
    for i in range(len(sat_positions)):
        r_vec = -sat_positions[i]
        r = np.linalg.norm(r_vec)
        acc = GM * r_vec / r**3
        sat_velocities[i] += acc * dt
        sat_positions[i] += sat_velocities[i] * dt

    # Update debris
    for i in range(len(debris_positions)):
        r_vec = -debris_positions[i]
        r = np.linalg.norm(r_vec)
        acc = GM * r_vec / r**3
        debris_velocities[i] += acc * dt
        debris_positions[i] += debris_velocities[i] * dt

    # Detect satellite-satellite collisions
    new_debris = []
    collided = set()
    for i in range(len(sat_positions)):
        for j in range(i+1, len(sat_positions)):
            if np.linalg.norm(sat_positions[i] - sat_positions[j]) < 5:
                collided.update([i, j])
                for _ in range(6):
                    vel = sat_velocities[i] + np.random.normal(0, 1.5, 2)
                    new_debris.append((sat_positions[i].copy(), vel))

    # Remove collided satellites
    if collided:
        mask = np.array([i not in collided for i in range(len(sat_positions))])
        sat_positions = sat_positions[mask]
        sat_velocities = sat_velocities[mask]
        sat_sizes = sat_sizes[mask]
        sat_colors = [sat_colors[i] for i in range(len(mask)) if mask[i]]

    # Add debris
    for pos, vel in new_debris:
        debris_positions.append(pos)
        debris_velocities.append(vel)
        debris_colors.append('red')
        debris_sizes.append(2)

    # Combine all object data
    all_pos = [*sat_positions] + debris_positions
    all_col = [*sat_colors] + debris_colors
    all_siz = list(sat_sizes) + debris_sizes

    object_counts.append(len(all_pos))
    scat.set_offsets(np.array(all_pos))
    scat.set_sizes(np.array(all_siz) * 10)
    scat.set_color(all_col)

    # Time counter
    sim_time = frame * dt
    ax.set_title(f"Time: {sim_time:.1f} s | Objects: {len(all_pos)}")

    return scat,

# --- Animation ---
ani = FuncAnimation(fig, update, frames=1000, interval=100, blit=True)
HTML(ani.to_jshtml())

In [ ]:
#2b
# 📊 Plot object count over time
plt.figure(figsize=(8, 4))
plt.plot(object_counts, color='red')
plt.title("Total Number of Orbiting Objects Over Time")
plt.xlabel("Time (frames)")
plt.ylabel("Number of Objects (Satellites + Debris)")
plt.grid(True)
plt.show()

In [ ]:
#2c
import numpy as np
import matplotlib.pyplot as plt

# Kessler Syndrome Simulation Function
def run_kessler_sim(initial_sats, collision_chance, debris_per_collision, cleanup_rate, years=50):
    satellites = [initial_sats]
    debris = [0]
    total_objects = [initial_sats]

    for year in range(1, years + 1):
        current_sats = satellites[-1]
        current_debris = debris[-1]
        current_total = current_sats + current_debris

        expected_collisions = collision_chance * (current_total ** 2) / 2
        expected_collisions = min(expected_collisions, current_sats)

        new_debris = expected_collisions * debris_per_collision
        debris_decay = current_debris * cleanup_rate

        next_sats = current_sats - expected_collisions
        next_debris = current_debris + new_debris - debris_decay
        next_total = next_sats + next_debris

        satellites.append(next_sats)
        debris.append(next_debris)
        total_objects.append(next_total)

    return satellites, debris, total_objects

# Timeframe
years = 50
years_range = list(range(0, years + 1))
threshold = 10000  # Kessler congestion threshold

# Scenario 1: No Cleanup, Dense Orbits
sat1, deb1, tot1 = run_kessler_sim(
    initial_sats=3000,
    collision_chance=1e-4,
    debris_per_collision=300,
    cleanup_rate=0.0,
    years=years
)

# Scenario 2: Active Cleanup
sat2, deb2, tot2 = run_kessler_sim(
    initial_sats=1500,
    collision_chance=5e-6,
    debris_per_collision=100,
    cleanup_rate=0.05,
    years=years
)

# Plotting side-by-side
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
fig.suptitle("Kessler Syndrome Comparison: Catastrophic vs Controlled")

# Scenario 1 Plot
axs[0].plot(years_range, sat1, label="Satellites", color="blue")
axs[0].plot(years_range, deb1, label="Debris", color="red")
axs[0].plot(years_range, tot1, label="Total Objects", color="black", linestyle="--")
axs[0].axhline(threshold, color='purple', linestyle=':', linewidth=2, label="Threshold (10,000)")
axs[0].set_title("No Cleanup")
axs[0].set_xlabel("Years")
axs[0].set_ylabel("Number of Objects in Orbit")
axs[0].grid(True)
axs[0].legend()

# Scenario 2 Plot
axs[1].plot(years_range, sat2, label="Satellites", color="blue")
axs[1].plot(years_range, deb2, label="Debris", color="red")
axs[1].plot(years_range, tot2, label="Total Objects", color="black", linestyle="--")
axs[1].axhline(threshold, color='purple', linestyle=':', linewidth=2, label="Threshold (10,000)")
axs[1].set_title("With Cleanup")
axs[1].set_xlabel("Years")
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
#3a

# This simulation models satellite collisions and orbital debris
# inspired by the Kessler Syndrome.

# Student Tasks
# - Change initial_satellites
# - Tweak debris_per_collision
# - Try different too_close_threshold values
# - Stop when debris > 100
# - Graph debris vs. time and write a hypothesis

# Simulation Parameters (Editable by Students)
initial_satellites = 30      # Number of satellites at start
debris_per_collision = 2     # Debris created per collision
too_close_threshold = 5     # Proximity threshold (in km)
max_debris_limit = 100       # Stop simulation at this many debris

import random
import matplotlib.pyplot as plt

# Each satellite or debris has a 2D position
def random_position():
    return [random.uniform(0, 100), random.uniform(0, 100)]

# Initialize satellites
satellites = [random_position() for _ in range(initial_satellites)]
debris = []
debris_count_over_time = []

# Simulation Loop
max_steps = 50  # You can increase this if needed
for step in range(max_steps):
    new_debris = []

    # Check each pair of satellites for collision
    for i in range(len(satellites)):
        for j in range(i + 1, len(satellites)):
            sat1 = satellites[i]
            sat2 = satellites[j]
            dx = sat1[0] - sat2[0]
            dy = sat1[1] - sat2[1]
            distance = (dx**2 + dy**2)**0.5
            if distance < too_close_threshold:
                # Collision occurred
                for _ in range(debris_per_collision):
                    new_debris.append(random_position())

    debris.extend(new_debris)
    debris_count_over_time.append(len(debris))

    # Stop if debris exceeds maximum
    if len(debris) > max_debris_limit:
        print(f"Simulation stopped at step {step} — debris exceeded limit ({len(debris)})")
        break

# --- Plot Debris Over Time ---
plt.figure(figsize=(8, 4))
plt.plot(debris_count_over_time, marker='o', linestyle='-', color='red')
plt.title('Orbital Debris Over Time')
plt.xlabel('Time Step')
plt.ylabel('Debris Count')
plt.grid(True)
plt.show()

# Hypothesis Prompt
print("Write Your Hypothesis:")
print("What caused the rapid increase in orbital debris?")
print("- Was there a chain reaction of collisions?")
print("- Did your threshold or debris-per-collision value contribute?")
print("- How might this relate to the Kessler Syndrome in real life?")



In [ ]:
#4a
import random
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# === CONFIGURATION ===
too_close_thresholds = {'LEO': 10, 'MEO': 15, 'GEO': 20}
debris_per_collision = 5
max_debris_limit = 500
max_steps = 100
leo_lifetime = 20

launch_cost = {'LEO': 2_000_000, 'MEO': 3_000_000, 'GEO': 5_000_000}
insurance_cost = 500_000
insurance_refund = 1_000_000
launch_budget = 100_000_000

orbital_layers = ['LEO', 'MEO', 'GEO']
layer_bounds = {'LEO': (0, 50), 'MEO': (50, 100), 'GEO': (100, 150)}

satellites = []
debris = []
debris_count_over_time = []
active_satellites_over_time = []

solar_activity_chance = 0.1
debris_decay_rate = 0.3

current_policy = 'no_policy'
step_counter = 0
risk_streak = 0
low_risk_goal = 10
gps_goal = 3
gps_launched = 0
missions_success = []

launched_layers = set()
launch_history = []
cascade_triggered = False
insurance_payouts = 0
expired_leo_sats = 0

satellite_types = ['Weather', 'Comms', 'GPS']
purpose_counts = {'Weather': 0, 'Comms': 0, 'GPS': 0}

# === RANDOMIZED MISSIONS CONFIG ===
all_missions = {
    'GPS_3_by_20': {
        'desc': 'Launch 3 GPS satellites by Step 20',
        'check': lambda: gps_launched >= gps_goal and step_counter >= 20,
    },
    '10x_LOW_RISK': {
        'desc': 'Maintain LOW risk for 10 consecutive steps',
        'check': lambda: risk_streak >= low_risk_goal,
    },
    'All_3_Layers': {
        'desc': 'Launch into all 3 orbits (LEO, MEO, GEO)',
        'check': lambda: len(launched_layers) == 3,
    },
    'Balanced_Deployment': {
        'desc': 'Launch at least 5 satellites into each orbit',
        'check': lambda: all(launch_history.count(layer) >= 5 for layer in orbital_layers),
    },
    'Insurance_Strategist': {
        'desc': 'Get 3 insurance payouts without exceeding medium risk',
        'check': lambda: insurance_payouts >= 3 and all(d < 150 for d in debris_count_over_time),
    },
    'Skip_Steps': {
        'desc': 'Skip launching for 5 total steps',
        'check': lambda: launch_history.count("none") >= 5,
    },
    'No_Cascades': {
        'desc': 'Avoid collision cascades entirely',
        'check': lambda: not cascade_triggered,
    },
    'Debris_Dropper': {
        'desc': 'Reduce debris below 50 after passing 200',
        'check': lambda: any(
            d < 50 for i, d in enumerate(debris_count_over_time)
            if max(debris_count_over_time[:i+1], default=0) > 200
        ),
    },
    'Danger_Survivor': {
        'desc': 'Survive 5 CRITICAL risk steps without exceeding max debris',
        'check': lambda: sum(d > 300 for d in debris_count_over_time) >= 5 and len(debris) <= max_debris_limit,
    },
    'Sustainable_Skies': {
        'desc': 'End with <100 debris and >10 satellites',
        'check': lambda: len(debris) < 100 and len(satellites) > 10,
    },
    'Natural_Retirement': {
        'desc': 'Let 3 LEO satellites expire naturally',
        'check': lambda: expired_leo_sats >= 3,
    },
    'Orbital_Streak': {
        'desc': 'Launch into same orbit 5 times in a row',
        'check': lambda: any(launch_history[i:i+5].count(launch_history[i]) == 5 for i in range(len(launch_history)-4)),
    },
    'GPS_Priority': {
        'desc': 'Keep GPS as ≥30% of active satellites',
        'check': lambda: len(satellites) > 0 and (sum(1 for s in satellites if s['purpose'] == 'GPS') / len(satellites)) >= 0.3,
    }
}

selected_missions = random.sample(list(all_missions.keys()), 3)
mission_status = {k: False for k in selected_missions}

# === UI SETUP ===
def update_mission_display():
    html = "<b>🎯 Missions:</b><br>"
    for key in selected_missions:
        status = "✅" if mission_status[key] else "⬜"
        html += f"{status} {all_missions[key]['desc']}<br>"
    missions_box.value = html

# === HELPERS ===
def random_position_in_layer(layer):
    x = random.uniform(0, 100)
    y_range = layer_bounds[layer]
    y = random.uniform(*y_range)
    return x, y

def random_satellite(layer, insured=False):
    pos = random_position_in_layer(layer)
    purpose = random.choice(satellite_types)
    return {
        'x': pos[0],
        'y': pos[1],
        'layer': layer,
        'purpose': purpose,
        'age': 0,
        'insured': insured
    }

def check_collision(s1, s2):
    if s1['layer'] != s2['layer']:
        return False
    dx = s1['x'] - s2['x']
    dy = s1['y'] - s2['y']
    distance = (dx**2 + dy**2)**0.5
    threshold = too_close_thresholds[s1['layer']]
    return distance < threshold

def simulate_step(launch_layer=None, insure=False):
    global step_counter, satellites, debris, launch_budget
    global gps_launched, risk_streak, insurance_payouts, expired_leo_sats, cascade_triggered

    if step_counter >= max_steps or len(debris) > max_debris_limit:
        print("⚠️ Simulation ended or max debris exceeded.")
        disable_buttons()
        show_plot()
        return

    print(f"\n--- Step {step_counter+1} ---")

    if launch_layer in orbital_layers:
        cost = launch_cost[launch_layer] + (insurance_cost if insure else 0)
        if launch_budget >= cost:
            sat = random_satellite(launch_layer, insured=insure)
            satellites.append(sat)
            purpose_counts[sat['purpose']] += 1
            launch_budget -= cost
            launch_history.append(launch_layer)
            launched_layers.add(launch_layer)
            if sat['purpose'] == 'GPS':
                gps_launched += 1
            print(f"🛰️ Launched {sat['purpose']} satellite into {launch_layer} (-${{cost:,}}, {'insured' if insure else 'uninsured'})")
        else:
            print("🚫 Not enough budget to launch.")
    elif launch_layer == "none":
        launch_history.append("none")
        print("⏭️ No satellite launched this step.")

    # === Decay old LEO satellites ===
    remaining = []
    for sat in satellites:
        sat['age'] += 1
        if sat['layer'] == 'LEO' and sat['age'] >= leo_lifetime:
            expired_leo_sats += 1
            purpose_counts[sat['purpose']] -= 1
            print(f"☠️ LEO satellite expired (age {sat['age']})")
        else:
            remaining.append(sat)
    satellites[:] = remaining

    # === Collisions ===
    new_debris = []
    collided = set()
    collision_count = 0
    for i in range(len(satellites)):
        for j in range(i + 1, len(satellites)):
            if check_collision(satellites[i], satellites[j]):
                collided.update([i, j])
                for _ in range(debris_per_collision):
                    x, y = random_position_in_layer(satellites[i]['layer'])
                    new_debris.append({'x': x, 'y': y, 'layer': satellites[i]['layer']})
                collision_count += 1

    if collision_count >= 3:
        cascade_triggered = True
        print("⚠️ Collision cascade started!")

    for idx in collided:
        sat = satellites[idx]
        if sat['insured']:
            launch_budget += insurance_refund
            insurance_payouts += 1
            print("💵 Insurance payout for lost satellite.")

    satellites = [s for idx, s in enumerate(satellites) if idx not in collided]
    debris.extend(new_debris)

    # === Solar activity decay ===
    if random.random() < solar_activity_chance:
        old_len = len(debris)
        debris = [d for d in debris if d['layer'] != 'LEO' or random.random() > debris_decay_rate]
        print(f"🌞 Solar storm! Debris reduced from {old_len} to {len(debris)}")

    # === Metrics + Risk ===
    debris_count_over_time.append(len(debris))
    active_satellites_over_time.append(len(satellites))
    step_counter += 1

    risk = "🟢 LOW"
    if len(debris) > 150:
        risk = "🟡 MEDIUM"
    if len(debris) > 300:
        risk = "🔴 CRITICAL"

    if risk == "🟢 LOW":
        risk_streak += 1
    else:
        risk_streak = 0

    print(f"📊 Debris: {len(debris)} | Satellites: {len(satellites)} | Budget: ${{launch_budget:,}} | Risk: {risk}")

    # === Check Missions ===
    for key in selected_missions:
        if not mission_status[key] and all_missions[key]['check']():
            mission_status[key] = True

    update_mission_display()

    if len(debris) > max_debris_limit or step_counter >= max_steps:
        end_simulation()

def end_simulation():
    disable_buttons()
    score = len(satellites) - len(debris)
    print(f"🏁 Final Score: {score} (Satellites - Debris)")
    print("--- Mission Results ---")
    for k in selected_missions:
        status = "✅" if mission_status[k] else "❌"
        print(f"{status} {all_missions[k]['desc']}")
    show_plot()

def disable_buttons():
    button_leo_ins.disabled = True
    button_meo_ins.disabled = True
    button_geo_ins.disabled = True
    button_leo.disabled = True
    button_meo.disabled = True
    button_geo.disabled = True
    button_none.disabled = True

def show_plot():
    plt.figure(figsize=(10, 4))
    plt.plot(debris_count_over_time, label='Debris Count', color='red')
    plt.plot(active_satellites_over_time, label='Active Satellites', color='green')
    plt.xlabel('Time Step')
    plt.ylabel('Count')
    plt.title('Simulation Outcome')
    plt.grid(True)
    plt.legend()
    plt.show()

# === BUTTONS ===
button_leo = widgets.Button(description="LEO ($2M)")
button_meo = widgets.Button(description="MEO ($3M)")
button_geo = widgets.Button(description="GEO ($5M)")
button_leo_ins = widgets.Button(description="LEO + Insurance ($2.5M)")
button_meo_ins = widgets.Button(description="MEO + Insurance ($3.5M)")
button_geo_ins = widgets.Button(description="GEO + Insurance ($5.5M)")
button_none = widgets.Button(description="Skip ❌")

button_leo.on_click(lambda b: simulate_step('LEO', insure=False))
button_meo.on_click(lambda b: simulate_step('MEO', insure=False))
button_geo.on_click(lambda b: simulate_step('GEO', insure=False))
button_leo_ins.on_click(lambda b: simulate_step('LEO', insure=True))
button_meo_ins.on_click(lambda b: simulate_step('MEO', insure=True))
button_geo_ins.on_click(lambda b: simulate_step('GEO', insure=True))
button_none.on_click(lambda b: simulate_step('none'))

missions_box = widgets.HTML()
update_mission_display()

display(widgets.VBox([
    missions_box,
    widgets.HBox([button_leo, button_meo, button_geo]),
    widgets.HBox([button_leo_ins, button_meo_ins, button_geo_ins]),
    button_none
]))

print("⬇️ Choose your launch. You can insure satellites or skip.")

In [ ]:
#4b
# Step 1: Install FFmpeg for saving animations
!apt-get install -y ffmpeg

# Step 2: Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

# === Configuration ===
np.random.seed(42)
num_fragments = 200
zones = np.random.choice(['LEO', 'MEO', 'GEO'], size=num_fragments, p=[0.6, 0.25, 0.15])
altitudes = {'LEO': 7000, 'MEO': 17000, 'GEO': 42164}  # km from Earth center
zone_colors = {'LEO': 'blue', 'MEO': 'green', 'GEO': 'orange'}
angular_velocities = {'LEO': 0.002, 'MEO': 0.001, 'GEO': 0.0005}  # radians/sec

# Assign orbital parameters
radii = np.array([altitudes[z] for z in zones])
omegas = np.array([angular_velocities[z] for z in zones])
initial_angles = np.random.uniform(0, 2 * np.pi, size=num_fragments)
inclinations = np.radians(np.random.uniform(-30, 30, size=num_fragments))  # random tilt
colors = [zone_colors[z] for z in zones]

# Earth sphere
earth_radius = 6371
u = np.linspace(0, 2 * np.pi, 50)
v = np.linspace(0, np.pi, 50)
x_earth = earth_radius * np.outer(np.cos(u), np.sin(v))
y_earth = earth_radius * np.outer(np.sin(u), np.sin(v))
z_earth = earth_radius * np.outer(np.ones(np.size(u)), np.cos(v))

# === Plot Setup ===
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(-50000, 50000)
ax.set_ylim(-50000, 50000)
ax.set_zlim(-50000, 50000)
ax.set_box_aspect([1, 1, 1])
ax.set_xlabel("X (km)")
ax.set_ylabel("Y (km)")
ax.set_zlabel("Z (km)")
ax.view_init(elev=25, azim=30)
ax.plot_surface(x_earth, y_earth, z_earth, color='lightblue', alpha=0.3, linewidth=0)

# Zone Labels
for z, alt in altitudes.items():
    ax.text(0, 0, alt, f"{z}", color=zone_colors[z], fontsize=10)

# Debris dots + trails
sc = ax.scatter([], [], [], s=15, alpha=0.8)
trail_len = 10
trail_lines = [ax.plot([], [], [], lw=0.5, color=zone_colors[z])[0] for z in zones]
trail_history = np.zeros((num_fragments, trail_len, 3))

# === Animation Update Function ===
def update(frame):
    t = frame * 100
    theta = initial_angles + omegas * t
    x = radii * np.cos(theta)
    y = radii * np.sin(theta)
    z = radii * np.sin(inclinations) * np.sin(theta)

    # Debris update
    sc._offsets3d = (x, y, z)
    sc.set_color(colors)

    # Trail update
    trail_history[:, :-1] = trail_history[:, 1:]
    trail_history[:, -1, 0] = x
    trail_history[:, -1, 1] = y
    trail_history[:, -1, 2] = z

    for i, line in enumerate(trail_lines):
        line.set_data(trail_history[i, :, 0], trail_history[i, :, 1])
        line.set_3d_properties(trail_history[i, :, 2])

    ax.view_init(elev=25, azim=30 + frame * 1.2)
    ax.set_title(f"Orbital Debris Rings (t = {t} s)", fontsize=14)
    return sc, *trail_lines

# === Generate Animation ===
anim = FuncAnimation(fig, update, frames=90, interval=120, blit=False)
anim.save("orbital_debris_rings_final.mp4", writer="ffmpeg", fps=15)

# === Download the video ===
from google.colab import files
files.download("orbital_debris_rings_final.mp4")


In [ ]:
#https://en.wikipedia.org/wiki/2024_YR4#:~:text=This%20was%20the%20second%2Dhighest,impact%20on%201%20March%202025.
#5a
import numpy as np
from datetime import datetime
#A dictionary (asteroid_data) stores simplified orbital parameters of 2024 YR4, such as:
#Eccentricity, semi-major axis, inclination, perihelion distance, etc.
#These are used as inputs for position and velocity calculations.
# Define the orbital elements of 2024 YR4 (as an example) for the collision risk calculation
asteroid_data = {
    'e': 0.6615483387208111,  # Eccentricity
    'a': 2.515868595636145,  # Semi-major axis in AU
    'q': 0.8514999057531932,  # Perihelion in AU
    'i': 3.408174626160702,  # Inclination in degrees
    'node': 271.365616491282,  # Longitude of ascending node in degrees
    'peri': 134.3613646975317,  # Argument of perihelion in degrees
    'M': 40.40255371661091,  # Mean anomaly in degrees
    'tp': 2460636.917563124088,  # Time of perihelion passage (Modified Julian Date)
    'period': 1457.573144717252,  # Orbital period in days
    'n': 0.2469858897337429,  # Mean motion (deg/day)
    'Q': 4.180237285519097,  # Aphelion distance in AU
}

# Define the current date and time (March 8, 2025)
current_time = datetime(2025, 3, 8)

# Function to calculate distance and relative velocity for the asteroid (simplified for this example)
def calculate_asteroid_position_velocity(data, time_since_perihelion):
    """Calculate the asteroid's position and velocity given the time since perihelion."""
    # Using simplified orbital mechanics (Keplerian orbit assumptions)
    mean_anomaly = (data['M'] + data['n'] * time_since_perihelion) % 360  # Mean anomaly
    # For simplicity, we will assume circular orbits (ignoring eccentricity for this simple model)
    # Calculate the distance based on semi-major axis for circular orbit
    distance = data['a']  # In AU
    velocity = np.sqrt((1 / distance) * (1 / data['a']))  # Simplified velocity for circular orbit
    return distance, velocity

# Function to estimate collision risk based on distance, relative velocity, and impact probability
def estimate_collision_risk(distance, relative_velocity, impact_probability, safe_distance=10.0, max_velocity=3.0):
    """Estimate the risk of collision with Earth based on distance, velocity, and impact probability."""
    # Adjust the risk based on the tracking confidence and impact probability
    if impact_probability > 0:
        # Calculate risk based on distance and velocity first
        collision_risk = 0.0
        if distance < safe_distance:
            velocity_magnitude = np.linalg.norm(relative_velocity)
            if velocity_magnitude > max_velocity:
                collision_risk = 1.0
            else:
                collision_risk = velocity_magnitude / max_velocity
        # Now factor in the impact probability (could be from initial tracking or a worst-case scenario)
        collision_risk *= impact_probability  # Weight the collision risk by the impact probability
        return collision_risk
    else:
        return 0.0  # No risk if probability is zero or unknown

# Example calculation (time since perihelion as an example)
time_since_perihelion = (current_time - datetime(2024, 11, 22)).days  # Days since perihelion on Nov 22, 2024

# Simplified asteroid position and velocity calculation (for demonstration purposes)
distance, velocity = calculate_asteroid_position_velocity(asteroid_data, time_since_perihelion)

# Example impact probability (say 3.1% as a worst-case scenario initially, now reduced)
impact_probability = 0.031  # 3.1% chance, adjusted over time

# Now, calculate the collision risk
collision_risk = estimate_collision_risk(distance, velocity, impact_probability)

# Display the result
print(f"Collision risk for asteroid 2024 YR4: {collision_risk:.4f}")